# Working with NV Aggregated Data

Pre-processing input data for a smoother upload experience of the state data to the WaDE 2.0 database.
Using geopandas to read in shp file, and coverting to WKT for ReportingUnit geometry.

#### Notes:
- NA

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

#Setting work directory, reading inputs, creating dataframe
workingDir = "G:/Shared drives/WaDE Data/Nevada/AggregatedAmounts/RawInputData"
os.chdir(workingDir)

# Pumpage Data

In [2]:
#CSV input file
basin2015Input = "StateInv_2015_BasinPumpage.csv"
basin2017Input = "StateInv_2017_BasinPumpage.csv"
county2015Input = "StateInv_2015_CountyPumpage.csv"
county2017Input = "StateInv_2015_CountyPumpage.csv"

In [3]:
# basin 2015
dfb15 = pd.read_csv(basin2015Input)
dfb15['Year'] = "2015"
dfb15['in_ReportingUnitType'] = "Basin"
print(len(dfb15))
dfb15.head(1)

256


,OID_,BasinID,BasinName,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,Shape_Length,Shape_Area,Year,in_ReportingUnitType
0,1,009,Long Valley,1245.0,0.0,0.0,6.0,0.0,0.0,1103.0,0.0,0.0,0.0,0,0.0,0.0,136,0.0,225645.674761,1.126936e+09,2015,Basin


In [4]:
# basin 2017
dfb17 = pd.read_csv(basin2017Input)
dfb17['Year'] = "2017"
dfb17['in_ReportingUnitType'] = "Basin"
print(len(dfb17))
dfb17.head(1)

256


,OID_,BasinID,BasinName,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Length,Shape_Area,Year,in_ReportingUnitType
0,1,009,Long Valley,297.0,0.0,0.0,6.0,0.0,0.0,155.0,0.0,0.0,0.0,0,0.0,0.0,136,0.0,{00000000-0000-0000-0000-000000000000},225645.674761,1.126936e+09,2017,Basin


In [5]:
# coutny 2015
dfc15 = pd.read_csv(county2015Input)
dfc15['Year'] = "2015"
dfc15['in_ReportingUnitType'] = "County"
print(len(dfc15))
dfc15.head(1)

17


,OID_,County,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Length,Shape_Area,Year,in_ReportingUnitType
0,1,Clark,98420.244,1363.22,70.58,5779.5,2784.16,4125.42,5216.0,2197.3,59698.16,977.0,0,10663.454,5410.45,21,114.0,{00000000-0000-0000-0000-000000000000},734866.608384,2.087734e+10,2015,County


In [6]:
# coutny 2017
dfc17 = pd.read_csv(county2017Input)
dfc17['Year'] = "2017"
dfc17['in_ReportingUnitType'] = "County"
print(len(dfc17))
dfc17.head(1)

17


,OID_,County,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Length,Shape_Area,Year,in_ReportingUnitType
0,1,Clark,98420.244,1363.22,70.58,5779.5,2784.16,4125.42,5216.0,2197.3,59698.16,977.0,0,10663.454,5410.45,21,114.0,{00000000-0000-0000-0000-000000000000},734866.608384,2.087734e+10,2017,County


### Basin Files

In [7]:
# Concatenate basin dataframes.
frames = [dfb15, dfb17]
dfbtemp = pd.concat(frames).reset_index(drop=True)
dfbtemp['in_ReportingUnitNativeID'] = dfbtemp['BasinID'].astype(str)
dfbtemp['in_ReportingUnitName'] = dfbtemp['BasinName']
dfbtemp = dfbtemp.drop(['OID_', 'BasinID', 'BasinName'], axis=1)
print(len(dfbtemp))
dfbtemp.head(3)

512


,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,Shape_Length,Shape_Area,Year,in_ReportingUnitType,GlobalID,in_ReportingUnitNativeID,in_ReportingUnitName
0,1245.00,0.0,0.0,6.0,0.0,0.0,1103.0,0.0,0.00,0.0,0,0.0,0.0,136,0.0,225645.674761,1.126936e+09,2015,Basin,NaN,009,Long Valley
1,3292.03,0.0,0.0,42.0,0.0,0.0,3124.0,0.0,59.33,0.0,0,46.7,0.0,20,0.0,233873.918113,1.516127e+09,2015,Basin,NaN,033B,Quinn River Valley
2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0,0.0,0.0,0,0.0,47416.631321,8.575338e+07,2015,Basin,NaN,012,Mosquito Valley


In [8]:
# need to transpose the data.
# create Temporary basin out dataframe
columnsList = ["Year", "in_ReportingUnitType", "in_ReportingUnitNativeID", "in_ReportingUnitName"]
dfbtemp2 = pd.DataFrame(columns=columnsList)
dfbtemp2 = dfbtemp[columnsList]

############################################

dfbtemp2 = dfbtemp2.assign(NV_BenUse='')
dfbtemp2 = dfbtemp2.assign(in_Amount='')
dfbtemp2 = dfbtemp2.assign(in_ReportingUnitType='')
dfBasinOut = pd.DataFrame() # dataframe to append to

############################################
columnsList = [
    'COM',
    'CON',
    'DOM',
    'ENV',
    'IND',
    'IRR',
    'MM',
    'MUN',
    'OTH',
    'PWR',
    'QM',
    'REC',
    'STK',
    'WLD']
lenList = len(columnsList)


############################################
for i in range(lenList):
    BenuseString = columnsList[i]
    dfbtemp2['NV_BenUse'] = BenuseString
    dfbtemp2['in_Amount'] = dfbtemp[columnsList[i]]
    dfbtemp2['in_ReportingUnitType'] = "Basin"
    dfBasinOut = dfBasinOut.append(dfbtemp2)
    
############################################

print(len(dfBasinOut.index))
dfBasinOut

7168


C:\Users\rjame\AppData\Local\Temp\ipykernel_21024\896595979.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfBasinOut = dfBasinOut.append(dfbtemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_21024\896595979.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfBasinOut = dfBasinOut.append(dfbtemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_21024\896595979.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfBasinOut = dfBasinOut.append(dfbtemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_21024\896595979.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfBasinOut = dfBasinOut.append(dfbtemp2)
C:\Users\rjame\AppData\Local

,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName,NV_BenUse,in_Amount
0,2015,Basin,009,Long Valley,COM,0.00
1,2015,Basin,033B,Quinn River Valley,COM,0.00
2,2015,Basin,012,Mosquito Valley,COM,0.00
3,2015,Basin,007,Swan Lake Valley,COM,0.00
4,2015,Basin,029,Pine Forest Valley,COM,4.48
...,...,...,...,...,...,...
507,2017,Basin,218,California Wash,WLD,0.00
508,2017,Basin,223,Gold Butte Area,WLD,0.00
509,2017,Basin,215,Black Mountains Area,WLD,0.00
510,2017,Basin,107,Smith Valley,WLD,0.00


### County Files

In [9]:
# Concatenate county dataframes.
frames = [dfc15, dfc17]
dfctemp = pd.concat(frames).reset_index(drop=True)
dfctemp['in_ReportingUnitNativeID'] = "WaDE_" + dfctemp['County']
dfctemp['in_ReportingUnitName'] = dfctemp['County']
dfctemp = dfctemp.drop(['OID_', 'County'], axis=1)
print(len(dfctemp))
dfctemp.head(3)

34


,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Length,Shape_Area,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName
0,98420.244000,1363.220000,70.58,5779.5,2784.16000,4125.420000,5216.00,2197.300000,59698.160000,977.0,0,10663.454000,5410.45,21,114.00,{00000000-0000-0000-0000-000000000000},734866.608384,2.087734e+10,2015,County,WaDE_Clark,Clark
1,12398.606126,34.929285,0.00,85.0,8.72708,72.303284,6544.00,2457.875809,944.925567,0.0,0,1882.355101,0.00,64,304.49,{00000000-0000-0000-0000-000000000000},483551.122120,9.870862e+09,2015,County,WaDE_Mineral,Mineral
2,113282.636403,2.000000,0.00,584.0,2217.00000,206.430000,99417.45,7695.166403,1111.000000,0.0,1177,349.590000,248.00,275,0.00,{00000000-0000-0000-0000-000000000000},632704.546310,1.428683e+10,2015,County,WaDE_Lander,Lander


In [10]:
# need to transpose the data.
# create Temporary county out dataframe
columnsList = ["Year", "in_ReportingUnitType", "in_ReportingUnitNativeID", "in_ReportingUnitName"]
dfctemp2 = pd.DataFrame(columns=columnsList)
dfctemp2 = dfctemp[columnsList]

############################################

dfctemp2 = dfctemp2.assign(NV_BenUse='')
dfctemp2 = dfctemp2.assign(in_Amount='')
dfctemp2 = dfctemp2.assign(in_ReportingUnitType='')
dfCountyOut = pd.DataFrame() # dataframe to append to

############################################
columnsList = [
    'COM',
    'CON',
    'DOM',
    'ENV',
    'IND',
    'IRR',
    'MM',
    'MUN',
    'OTH',
    'PWR',
    'QM',
    'REC',
    'STK',
    'WLD']
lenList = len(columnsList)


############################################
for i in range(lenList):
    BenuseString = columnsList[i]
    dfctemp2['NV_BenUse'] = BenuseString
    dfctemp2['in_Amount'] = dfctemp[columnsList[i]]
    dfctemp2['in_ReportingUnitType'] = "County"
    dfCountyOut = dfCountyOut.append(dfctemp2)
    
############################################

print(len(dfCountyOut.index))
dfCountyOut

476


C:\Users\rjame\AppData\Local\Temp\ipykernel_21024\1428374340.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCountyOut = dfCountyOut.append(dfctemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_21024\1428374340.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCountyOut = dfCountyOut.append(dfctemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_21024\1428374340.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCountyOut = dfCountyOut.append(dfctemp2)
C:\Users\rjame\AppData\Local\Temp\ipykernel_21024\1428374340.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCountyOut = dfCountyOut.append(dfctemp2)
C:\Users\rjame\A

,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName,NV_BenUse,in_Amount
0,2015,County,WaDE_Clark,Clark,COM,1363.220000
1,2015,County,WaDE_Mineral,Mineral,COM,34.929285
2,2015,County,WaDE_Lander,Lander,COM,2.000000
3,2015,County,WaDE_Churchill,Churchill,COM,767.920000
4,2015,County,WaDE_Eureka,Eureka,COM,14.000000
...,...,...,...,...,...,...
29,2017,County,WaDE_Lincoln,Lincoln,WLD,59.980000
30,2017,County,WaDE_Nye,Nye,WLD,5.000000
31,2017,County,WaDE_Pershing,Pershing,WLD,6.970000
32,2017,County,WaDE_Storey,Storey,WLD,1.000000


In [11]:
# Concatenate temp basin and county dataframes.
frames = [dfBasinOut, dfCountyOut]
dfout = pd.concat(frames).reset_index(drop=True)
print(len(dfout))
dfout

7644


,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName,NV_BenUse,in_Amount
0,2015,Basin,009,Long Valley,COM,0.00
1,2015,Basin,033B,Quinn River Valley,COM,0.00
2,2015,Basin,012,Mosquito Valley,COM,0.00
3,2015,Basin,007,Swan Lake Valley,COM,0.00
4,2015,Basin,029,Pine Forest Valley,COM,4.48
...,...,...,...,...,...,...
7639,2017,County,WaDE_Lincoln,Lincoln,WLD,59.98
7640,2017,County,WaDE_Nye,Nye,WLD,5.00
7641,2017,County,WaDE_Pershing,Pershing,WLD,6.97
7642,2017,County,WaDE_Storey,Storey,WLD,1.00


In [12]:
# TimeframeStart & TimeframeEnd

dfout['in_TimeframeStart'] = '01/01/' + dfout['Year'].astype(str)
dfout['in_TimeframeEnd'] = '12/31/' + dfout['Year'].astype(str)
dfout

,Year,in_ReportingUnitType,in_ReportingUnitNativeID,in_ReportingUnitName,NV_BenUse,in_Amount,in_TimeframeStart,in_TimeframeEnd
0,2015,Basin,009,Long Valley,COM,0.00,01/01/2015,12/31/2015
1,2015,Basin,033B,Quinn River Valley,COM,0.00,01/01/2015,12/31/2015
2,2015,Basin,012,Mosquito Valley,COM,0.00,01/01/2015,12/31/2015
3,2015,Basin,007,Swan Lake Valley,COM,0.00,01/01/2015,12/31/2015
4,2015,Basin,029,Pine Forest Valley,COM,4.48,01/01/2015,12/31/2015
...,...,...,...,...,...,...,...,...
7639,2017,County,WaDE_Lincoln,Lincoln,WLD,59.98,01/01/2017,12/31/2017
7640,2017,County,WaDE_Nye,Nye,WLD,5.00,01/01/2017,12/31/2017
7641,2017,County,WaDE_Pershing,Pershing,WLD,6.97,01/01/2017,12/31/2017
7642,2017,County,WaDE_Storey,Storey,WLD,1.00,01/01/2017,12/31/2017


In [13]:
# For creating BeneficialUse
BeneficialUseDict = {
    "COM" : "Commercial",
    "CON" : "Construction",
    "DOM" : "Domestic",
    "ENV" : "Environmental",
    "IND" : "Industrial",
    "IRR" : "Irrigation",
    "MM" : "Mining and Milling",
    "MUN" : "Municipal",
    "PWR" : "Power",
    "QM" : "Quasi-Municipal",
    "REC" : "Recreational",
    "STK" : "Stockwater",
    "WLD" : "Wildlife"}
def assignBeneficialUse(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        String1 = colrowValue.strip()
        try:
            outList = BeneficialUseDict[String1]
        except:
            outList = "Unspecified"

    return outList

dfout['in_BeneficialUseCategory'] = dfout.apply(lambda row: assignBeneficialUse(row['NV_BenUse']), axis=1)
dfout['in_BeneficialUseCategory'].unique()

array(['Commercial', 'Construction', 'Domestic', 'Environmental',
       'Industrial', 'Irrigation', 'Mining and Milling', 'Municipal',
       'Unspecified', 'Power', 'Quasi-Municipal', 'Recreational',
       'Stockwater', 'Wildlife'], dtype=object)

In [14]:
# in_VariableSpecificCV Info
dfout['in_VariableSpecificCV'] = "Withdrawal_Annual_" + dfout['in_BeneficialUseCategory'] + "_Groundwater"
dfout['in_VariableSpecificCV'].unique()

array(['Withdrawal_Annual_Commercial_Groundwater',
       'Withdrawal_Annual_Construction_Groundwater',
       'Withdrawal_Annual_Domestic_Groundwater',
       'Withdrawal_Annual_Environmental_Groundwater',
       'Withdrawal_Annual_Industrial_Groundwater',
       'Withdrawal_Annual_Irrigation_Groundwater',
       'Withdrawal_Annual_Mining and Milling_Groundwater',
       'Withdrawal_Annual_Municipal_Groundwater',
       'Withdrawal_Annual_Unspecified_Groundwater',
       'Withdrawal_Annual_Power_Groundwater',
       'Withdrawal_Annual_Quasi-Municipal_Groundwater',
       'Withdrawal_Annual_Recreational_Groundwater',
       'Withdrawal_Annual_Stockwater_Groundwater',
       'Withdrawal_Annual_Wildlife_Groundwater'], dtype=object)

# Shapefile Data

In [15]:
# Basin Shapefile input
basinShape = gpd.read_file('shapefile/NVBasinShapefile.shp', crs="EPSG:4326")
print(len(basinShape))
basinShape.head(1)

256


,BasinID,BasinName,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Leng,Shape_Le_1,Shape_Area,geometry
0,009,Long Valley,297.0,0.0,0.0,6.0,0.0,0.0,155.0,0.0,0.0,0.0,0,0.0,0.0,136,0.0,{D8338F68-4281-4E70-A4F9-F2B3CC075F52},225645.674761,2.297901,0.121633,"POLYGON ((-119.70599 41.96602, -119.70558 41.9..."


In [16]:
columnsList = ['in_ReportingUnitNativeID', 'RU_Name', 'RU_Type', 'geometry']
dfNVBshape = pd.DataFrame(columns=columnsList)
dfNVBshape['in_ReportingUnitNativeID'] = basinShape['BasinID'].astype(str)
dfNVBshape['RU_Name'] = basinShape['BasinName']
dfNVBshape['RU_Type'] = "Basin"
dfNVBshape['geometry'] = basinShape['geometry']
dfNVBshape = dfNVBshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfNVBshape.head(3)

,in_ReportingUnitNativeID,RU_Name,RU_Type,geometry
0,009,Long Valley,Basin,"POLYGON ((-119.70599 41.96602, -119.70558 41.9..."
1,033B,Quinn River Valley,Basin,"POLYGON ((-118.09829 41.99750, -118.09809 41.9..."
2,012,Mosquito Valley,Basin,"POLYGON ((-119.73707 41.87237, -119.73741 41.8..."


In [17]:
# Coutny Shapefile input
countyShape = gpd.read_file('shapefile/NVCountyShapefile.shp', crs="EPSG:4326")
print(len(countyShape))
countyShape.head(1)

17


,County,Total,COM,CON,DOM,ENV,IND,IRR,MM,MUN,OTH,PWR,QM,REC,STK,WLD,GlobalID,Shape_Leng,Shape_Le_1,Shape_Area,geometry
0,Clark,96196.8908,1297.0,438.7608,5746.0,3240.16,3238.25,6209.0,2989.16,57176.77,0.0,0,10203.34,5494.45,21,143.0,{083692DA-2B8B-47BD-8860-0E2BE4C35B1D},734866.608384,7.358416,2.092057,"POLYGON ((-115.89694 36.84209, -115.84021 36.8..."


In [18]:
columnsList = ['in_ReportingUnitNativeID', 'RU_Name', 'RU_Type', 'geometry']
dfNVCshape = pd.DataFrame(columns=columnsList)
dfNVCshape['in_ReportingUnitNativeID'] = "WaDE_" + countyShape['County']
dfNVCshape['RU_Name'] = countyShape['County']
dfNVCshape['RU_Type'] = "County"
dfNVCshape['geometry'] = countyShape['geometry']
dfNVCshape = dfNVCshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfNVCshape.head(3)

,in_ReportingUnitNativeID,RU_Name,RU_Type,geometry
0,WaDE_Clark,Clark,County,"POLYGON ((-115.89694 36.84209, -115.84021 36.8..."
1,WaDE_Mineral,Mineral,County,"POLYGON ((-119.15700 38.41474, -119.12599 38.4..."
2,WaDE_Lander,Lander,County,"POLYGON ((-117.80680 39.14538, -117.80386 39.1..."


In [19]:
# Concatenate basin and county shape dataframes together.
frames = [dfNVBshape, dfNVCshape]
dfAllShape = pd.concat(frames).reset_index(drop=True)
dfAllShape

,in_ReportingUnitNativeID,RU_Name,RU_Type,geometry
0,009,Long Valley,Basin,"POLYGON ((-119.70599 41.96602, -119.70558 41.9..."
1,033B,Quinn River Valley,Basin,"POLYGON ((-118.09829 41.99750, -118.09809 41.9..."
2,012,Mosquito Valley,Basin,"POLYGON ((-119.73707 41.87237, -119.73741 41.8..."
3,007,Swan Lake Valley,Basin,"POLYGON ((-119.33079 41.87199, -119.32664 41.8..."
4,029,Pine Forest Valley,Basin,"POLYGON ((-118.40997 41.88370, -118.39837 41.8..."
...,...,...,...,...
268,WaDE_Lincoln,Lincoln,County,"POLYGON ((-115.89447 36.86172, -115.89597 37.0..."
269,WaDE_Pershing,Pershing,County,"POLYGON ((-119.33871 40.15761, -119.33822 40.3..."
270,WaDE_Storey,Storey,County,"POLYGON ((-119.71408 39.25918, -119.70125 39.2..."
271,WaDE_Douglas,Douglas,County,"POLYGON ((-120.00342 39.11269, -119.95785 39.1..."


### Inspect Output Data & Export

In [20]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfout.dtypes)

Year                         object
in_ReportingUnitType         object
in_ReportingUnitNativeID     object
in_ReportingUnitName         object
NV_BenUse                    object
in_Amount                   float64
in_TimeframeStart            object
in_TimeframeEnd              object
in_BeneficialUseCategory     object
in_VariableSpecificCV        object
dtype: object


In [21]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfAllShape.dtypes)

in_ReportingUnitNativeID      object
RU_Name                       object
RU_Type                       object
geometry                    geometry
dtype: object


In [22]:
# Export out to CSV.
dfout.to_csv('P_nvAggMaster.csv', index=False) # The output.
dfAllShape.to_csv('P_nvGeometry.csv', index=False) # The output geometry.